In [ ]:
#The streaming api is quite different from the REST api because the REST api is used to pull data from twitter
#But the streaming api pushes messages to a persistent session.
#This allows the streaming api to download more data in real time than could be done using the REST API.
#In Tweepy, an instance of tweepy.Stream establishes a streaming session and routes messages to StreamListener instance.
#The on_data method of a stream listener receives all messages and calls functions according to the message type. 
#The default StreamListener can classify most common twitter messages and routes them to appropriately named methods


#Steps to Follow

#Create a class inheriting from StreamListener
#Using that class create a Stream object
#Connect to the Twitter API using the Stream.

In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sqlite3
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from unidecode import unidecode
import time

analyzer = SentimentIntensityAnalyzer()

#consumer key, consumer secret, access token, access secret.


with open('credentials.json') as f:
    data = json.load(f)
    
ckey=data['ckey']
csecret=data['csecret']
atoken=data['atoken']
asecret=data['asecret']

conn = sqlite3.connect('twitter.db')
c = conn.cursor()

def create_table():
    try:
        c.execute("CREATE TABLE IF NOT EXISTS sentiment(unix REAL, tweet TEXT, sentiment REAL)")
        c.execute("CREATE INDEX fast_unix ON sentiment(unix)")
        c.execute("CREATE INDEX fast_tweet ON sentiment(tweet)")
        c.execute("CREATE INDEX fast_sentiment ON sentiment(sentiment)")
        conn.commit()
    except Exception as e:
        print(str(e))
        
create_table()



class listener(StreamListener):

    def on_data(self, data):
        try:
            data = json.loads(data)
            tweet = unidecode(data['text'])
            time_ms = data['timestamp_ms']
            vs = analyzer.polarity_scores(tweet)
            sentiment = vs['compound']
            print(time_ms, tweet, sentiment)
            c.execute("INSERT INTO sentiment (unix, tweet, sentiment) VALUES (?, ?, ?)",
                  (time_ms, tweet, sentiment))
            conn.commit()

        except KeyError as e:
            print(str(e))
        return(True)

    def on_error(self, status):
        print(status)


while True:

    try:
        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        twitterStream = Stream(auth, listener())
        twitterStream.filter(track=["trump","usa","h1b","visa","covid-19","virus"])
    except Exception as e:
        print(str(e))
        time.sleep(5)

1593558271958 RT @yogagenie: Trump campaign cancels AL rally as Covid cases surge https://t.co/d2EmyzJDCb -0.2263
1593558271958 RT @BillKristol: The rest of us can be angry at China and angry at Trump. -0.765
1593558271942 What do you think?  Will he quit the race?
@President Trump is like a dead man golfing. So will he drop out of the... https://t.co/3DqVnAcxqC -0.673
1593558272030 RT @alvaro_delgado: Que miserable el gobierno de Queretaro: El secretario de Salud del panista @panchdominguez_, @JULIOCE56737355, confiesa... -0.4939
1593558272003 RT @MatthewLillard: This is the meanest thing the internet has EVER said to ANYONE, ever.  

And I'm available. https://t.co/owsWi8FTbu 0.0
1593558272022 RT @BeeJaySee: @Inevitable_ET https://t.co/X9QXE9KcY8 0.0
1593558272067 RT @AndyOstroy: So let's get this straight: @realdonaldtrump is either a racist POS who retweets "White Power"...or...he's a dumbass POS wh... -0.7561
1593558272040 RT @NevadaElJefe: Jill Biden: 'A Lot of Republicans Are Go

1593558272789 RT @AngrierWHStaff: Well shit. 

On the bright side, maybe Rand Paul will get it again. 0.1027
1593558272783 RT @YodaSapiens_29: Lista dos 44 opara marcarmos seus nomes na historia como gente que tentou criminalizar a liberdade de expressao https:/... 0.0
1593558272817 RT @CMOIreland: I encourage anyone who has received an invitation to participate in the #COVID19 Antibody Study (SCOPI) to respond.

It inv... 0.5106
1593558272814 RT @PamKeithFL: I call on Donald Trump to resign his presidency immediately. -0.34
1593558272811 RT @VickyisGrateful: Did anyone watch Inside Edition last night? They sent a crew to Trump Hotel in NY &amp; sprayed some invisible stuff on th... 0.0
1593558272832 RT @Mar_Cherbonneau: "La cuarentena es una verguenza. Eso lo que hay que hacer es que la gente se proteja y salga a trabajar"  Francisco Ba... 0.0
1593558272832 RT @KC_me_rollin: Wow... wonder why myrtle beach is a hot spot for COVID-19 right now!! https://t.co/AyY7j0aig1 0.658
15935582728

1593558273861 @HKrassenstein @realDonaldTrump Strange, other countries are doing so much better that the USA! I wonder if their l... https://t.co/gPmsTUMiww 0.396
1593558273815 Judge blocks tell-all by Trump niece - for now https://t.co/zPfZAtXke4 https://t.co/GA5oXyiqcm https://t.co/dbOMbGcJvb -0.2263
1593558273877 @Trump @EricTrump @TrumpHotels @CNTraveler They don't change the sheets! 0.0
1593558273930 RT @realDonaldTrump: As I watch the Pandemic spread its ugly face all across the world, including the tremendous damage it has done to the... -0.7778
1593558273901 Can't wait for swine flu part two, the first one made me shit myself in front of my crush in 9th grade :) -0.5859
1593558273936 RT @Justin12393LEE: @realDonaldTrump Trump is in office.  Putin is in power. 

#RussianBounty
 https://t.co/T3nUebxSfS 0.0
1593558273954 RT @RepsForBiden: There is nothing that drives Trump NUTS more than Republican &amp; Conservatives who put #CountryOverParty by supporting Bide... -0.0343
1593558

1593558274955 RT @harrylitman: Trump has maneuvered himself into a perfect corner with this mask dynamic. If he starts wearing one, he's a loser; if he c... 0.4404
1593558274969 @AP The virus is an opportunist, it goes when there is least resistance. Initially it took advantage of popular pop... https://t.co/gszp2n7Zei 0.6124
1593558274971 RT @jamieleecurtis: One person I am thinking about wears a mask to terrify people (me ESPECIALLY) but there's nothing more terrifying than... 0.4892
1593558274984 RT @SenSchumer: It's been 5 days since media reports informed the world of Putin's alleged bounty program. What has Pres. Trump done?
 
He... 0.0
1593558274879 via @NewsCorpse Trump Has Become a Parody of Himself with Laughable Reelection Campaign https://t.co/inQzJ839bl https://t.co/tTgSM75MMe 0.0516
1593558274978 Stop trying to warmonger China over a global disease -0.7269
1593558275007 RT @SethAbramson: BREAKING NEWS from THE ASSOCIATED PRESS: Trump Personally Briefed on Kremlin Murder P

1593558275889 Que mal como trata a este latino sudaca LGBT la tierra del fascista opresor de minorias de Trump.... 0.0
1593558275940 RT @streetnoodle: @ralphreed @PeteButtigieg @VP Oh, how you deceive. @PeteButtigieg asked a question. You twist it into a claim.

I am a Ch... -0.4019
1593558275912 @wendyp4545 @sie2sienna Their historically mode of operations: accuse the Republicans, mainly Trump, for what they... https://t.co/jo7OD56OMV -0.2023
1593558275957 RT @DWUhlfelderLaw: Florida has had a 604% increase in new COVID-19 cases in the past 30 days.  It ranks #1 ahead of Arizona at 443% increa... 0.3182
1593558275979 @bebe1969 He's sucks just teeth a lot. Nasty habit. Almost as bad a Trump's sniffing. -0.8527
1593558275988 RT @stevenspohn: On June 11th, Michael Hickson, a quadriplegic black man with COVID-19 was killed by a hospital in Austin Texas.

Doctors d... -0.6705
1593558276018 RT @TimOBrien: Trump stood next to a sculpture in the Oval Office and told Fox it depicts former pres

...

1593558285115 RT @HKrassenstein: BREAKING:  Joe Biden to Donald Trump moments ago: 

"Fix the PPE issue for healthcare workers before you tee-off for ano... 0.0
'text'
1593558285683 RT @ryanjreilly: "[REDACTED] replied 'fuck, fuck, I knew you guys would come, it was so fucked up.' [REDACTED] explained that no one on the... -0.9415
1593558285701 RT @PaulDFuchsMD1: Children. Political Pawns in the COVID-19 Fearmongering. 
The decision to close schools was never based on science. The... 0.0
1593558285719 RT @realDonaldTrump: As I watch the Pandemic spread its ugly face all across the world, including the tremendous damage it has done to the... -0.7778
1593558285753 RT @drdavidsamadi: I refuse to spread fear or paranoia where it's not needed.

We have more tools to defeat this virus than we did before.... -0.8646
1593558285756 RT @Elise_Jordan: My friend Dan served in the toughest combat zones in Iraq and Afghanistan and today saves lives as an ER doc. He's unspar... 0.128
1593558285767 RT

1593558286682 RT @Diego5Dario: Ano to UAAP?  0.0
1593558286689 RT @Elpetro14: Vayan a lavarse ese culo los dos y el hijo de puta de borrell que le vaya a besar el culo al Rey y almagro a trump 0.0
1593558286736 RT @curandero_el: Leicester, Reino Unido 
El centro de la ciudad hace una semana; ahora hay un brote de virus corona en todo el area y na... 0.0
1593558286736 RT @QDigitalArmy1: So why was the Queen talking with Trump?  0.0
1593558286719 Lockdown again . 0.0
1593558286759 RT @InternetHippo: (rolling up my sleeves to do a viral post) the corona virus must be a man...due to how bad it is -0.5423
1593558286768 RT @CountryTisOThee: Because we need to Remember our BEST FRIENDS

On the Ninth Day...

God Made A Dog

(with a nod to the fabulous @merlin... 0.9356
1593558286795 @bfrmilgrau @Botafogo Usa a tag #AssinaComBotafogoKalou 0.0
1593558286790 @JNAlexandratos @RepCohen She waited too long with Trump. Should have been impeached sooner. 0.0
1593558286808 RT @realDonaldTrump: As I wat

His administration knew our troops were in danger and did nothing. These servic... -0.7717
1593558287807 Time will tell 0.0
1593558287856 RT @LisaMei62: LMAO @ all the pearl-clutching over POTUS NOT wearing a mask. They say even if he's tested daily and doesn't have the virus... 0.6841
1593558287825 RT @Albagarnie: Estamos en el momento de la serie en el que la cosa que paso en el primer capitulo y quedo olvidada vuelve de pronto 0.0
1593558287877 RT @HaMeturgeman: Whether or not you kill the baby in your womb is "a personal choice and between you and your doctor" but whether or not y... 0.3333
1593558287888 RT @AmeliaHouchins: Trump committed treason. 
Trump committed treason. 
Trump committed treason. 
Trump committed treason. 
Trump committed... -0.4767
1593558287889 RT @ProjectLincoln: 19% Satisfaction within the Republican Party. Thank you!

https://t.co/UiMCdGuwb5 https://t.co/sttAgBwQM4 0.8122
1593558287881 RT @shadygrooove: Shew shew Chinese flu!! 

"We'll have to prepare for t

1593558288948 RT @wendydavis: It is unconscionable that any American President would allow what was essentially a "murder for hire" scheme against Americ... -0.5859
1593558288926 @CBSSportsRadio @mooretosay @ZachGelb @usatodaysports @JarrettBell @fangraphs @jay_jaffe July the fourth is looking... https://t.co/Os6ihzbZM3 0.0
1593558288888 RT @Mar_Cherbonneau: "La cuarentena es una verguenza. Eso lo que hay que hacer es que la gente se proteja y salga a trabajar"  Francisco Ba... 0.0
1593558288981 RT @LuckieKathy: 510 DAYS SOBER AND I'LL BE DAMNED IF TRUMP IS GOING TO MAKE ME DRINK AGAIN... -0.516
1593558288993 RT @SteveRustad1: What if Obama had known for a year about a Russian bounty on US troops &amp; done nothing?

MAGA's would have howled with fur... 0.0
1593558289048 RT @NWPinPDX: I do not understand how anyone who has served in the military or has seen a loved one sent over seas to defend our nation can... 0.5994
1593558289046 RT @matthewamiller: Maybe we should tell Trump the Rus

KeyboardInterrupt: 

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

vs = analyzer.polarity_scores('Hi I am happy today')
print(vs)

{'neg': 0.0, 'neu': 0.519, 'pos': 0.481, 'compound': 0.5719}


In [6]:
from textblob import TextBlob

In [8]:
tb = TextBlob('Hi I am happy today')
print(tb.sentiment)

Sentiment(polarity=0.8, subjectivity=1.0)


In [9]:
dir(TextBlob)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cmpkey',
 '_compare',
 '_create_sentence_objects',
 '_strkey',
 'analyzer',
 'classify',
 'correct',
 'detect_language',
 'ends_with',
 'endswith',
 'find',
 'format',
 'index',
 'join',
 'json',
 'lower',
 'ngrams',
 'noun_phrases',
 'np_counts',
 'np_extractor',
 'parse',
 'parser',
 'polarity',
 'pos_tagger',
 'pos_tags',
 'raw_sentences',
 'replace',
 'rfind',
 'rindex',
 'sentences',
 'sentiment',
 'sentiment_assessments',
 'serialized',
 'split',
 'starts_with',
 'startswith',
 'strip',
 'subjectivity',
 'tags',
 'title',
 'to_json',
 'tokenize'